<a href="https://colab.research.google.com/github/Ismail-Armutcu/BlockChain-CryptoCurrencyTechnologies/blob/main/Block%20Chain%20Analysis/BitcoinBlockAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import requests
from pprint import pprint  # Import pprint for pretty printing
import json

def get_block_details(block_hash):
    url = f"https://blockchain.info/rawblock/{block_hash}"
    response = requests.get(url)
    block = response.json()  # Convert the JSON response to a dictionary
    #block = response
    return block

# Example usage
block_hash = '00000000000000e25266d700ea597112a7adf393802db132f1ae7caee7313bdd'  # Replace with the block hash you are interested in
block_details = get_block_details(block_hash)

pprint(block_details)  # Prints the details of the block
#print(block_details.items)


Görüntülenen çıkış son 5000 satıra kısaltıldı.
                  'value': 1233576}],
         'relayed_by': '0.0.0.0',
         'size': 374,
         'time': 1369277732,
         'tx_index': 313170226161459,
         'ver': 1,
         'vin_sz': 2,
         'vout_sz': 2,
         'weight': 1496},
        {'block_height': 237439,
         'block_index': 237439,
         'double_spend': False,
         'fee': 100000,
         'hash': 'b80b5a4cc07bbcbc699a837fa8a86d73d64aa6d166437ea24e7b8aa7d09741e7',
         'inputs': [{'index': 0,
                     'prev_out': {'addr': '1dice7W2AicHosf5EL3GFDUVga7TgtPFn',
                                  'n': 0,
                                  'script': '76a91406f1b66fc9e59a7b4554cf2e6094032cd9ee45c488ac',
                                  'spending_outpoints': [{'n': 0,
                                                          'tx_index': 8136604989583695}],
                                  'spent': True,
                                  'tx_i

In [43]:
#for item in block_details.items():
 # print(item)

transactions = block_details.get("tx")

tx0 = transactions[1]
"""i=0
for tx in transactions:
  input_addresses = [input_['prev_out'].get('addr') for input_ in tx['inputs']]
  output_addresses = [output['addr'] for output in tx['out']]

  print(i,f"Input Addresses{len(input_addresses)}:", input_addresses," Output Addresses:", output_addresses)
  i=i+1"""

scripts = []
for tx in transactions:
  for input_ in tx['inputs']:
    scripts.append(input_['script'])
    scripts.append(input_['prev_out'].get('script'))
  for output_ in tx['out']:
    scripts.append(output_['script'])



In [25]:

def get_op_return_data(txid):
    url = f"https://blockchain.info/rawtx/{txid}"
    response = requests.get(url)
    tx_data = response.json()
    pprint(tx_data)
    for output in tx_data['out']:
        if 'script' in output and output['script'].startswith('6a'):
            hex_data = output['script'][4:]  # Skip the first 4 characters which are '6a' followed by the length byte
            try:
                print(bytes.fromhex(hex_data).decode('utf-8'))
            except:
                print("Non-UTF8 data or binary data")

# Example transaction ID with OP_RETURN
transaction_id = '65f9a0c1834c3c1815ae6822ffa745132b23bf0e4ce34c86a1848ea5daa96644'
get_op_return_data(transaction_id)



{'block_height': 237439,
 'block_index': 237439,
 'double_spend': False,
 'fee': 0,
 'hash': '65f9a0c1834c3c1815ae6822ffa745132b23bf0e4ce34c86a1848ea5daa96644',
 'inputs': [{'index': 0,
             'prev_out': {'addr': '1BzEsex7nG9mMwybwjpx4Q1wb1P6qBjHnH',
                          'n': 1,
                          'script': '76a91478848c6dbb095040ea855ee1856975d90686826088ac',
                          'spending_outpoints': [{'n': 0,
                                                  'tx_index': 2406647265014224}],
                          'spent': True,
                          'tx_index': 6383980498047237,
                          'type': 0,
                          'value': 1071023289},
             'script': '493046022100e8d9842e71cc243ea7404ab47e6d3d513fd298f071e32c66cce21cd747681702022100ee69c2329f3d1562afbf0c8f5310e04ca71bab3b0948da832e9a08f790b8b7c901410484a435cf7540fcf0efd056a9c60be4438488911ef293f5e49061ff3caa4318edb43f71810d37140b2747204b80eb6cea5c0ba1791fb1cad566a547d7

In [52]:
import binascii

def extract_printable_ascii(hex_string):
    # Convert the hex string to bytes
    byte_data = binascii.unhexlify(hex_string)

    # Define ASCII printable character ranges (space to ~)
    printable_ascii_range1 = (65, 90)
    printable_ascii_range2 = (97, 122)
    # Buffer to collect printable characters
    result = []
    current_printable_sequence = []

    # Iterate over each byte and check if it is a printable ASCII character
    for byte in byte_data:
        if printable_ascii_range1[0] <= byte <= printable_ascii_range1[1] or printable_ascii_range2[0] <= byte <= printable_ascii_range2[1]:
            current_printable_sequence.append(chr(byte))
        else:
            # If the current byte is not printable and there is an ongoing sequence,
            # save it if it has two or more characters
            if len(current_printable_sequence) > 1:
                result.append(''.join(current_printable_sequence))
            current_printable_sequence = []

    # Check if there is a remaining printable sequence at the end of the data
    if len(current_printable_sequence) > 1:
        result.append(''.join(current_printable_sequence))
    if(len(result)> 0 ):
      return result

# Example Usage
hex_string = '76a914069de9446799ffc2177e1c886c5aef55feea309e88ac'
printable_parts = extract_printable_ascii(hex_string)
print(printable_parts)

for sc in scripts:
  print(extract_printable_ascii(sc))


['Dg', 'lZ']
['Mined', 'by', 'BTC', 'Guild']
['AfK']
['qo', 'Uu', 'lu']
['kF', 'Ix']
None
['kF', 'Ix']
['MU', 'ao', 'WK', 'fD', 'em']
['YZ']
['MT', 'vH', 'uT']
['gZ']
None
None
['BS', 'dr', 'br', 'wZZ']
['Wp']
['VZ']
['Wp']
['cY', 'ljpB', 'dV', 'mW', 'dc', 'Lx', 'kZ', 'vI']
['gR']
['FJ', 'nU', 'HS', 'Lx', 'kZ', 'vI']
['gR']
None
['gR']
['gef', 'IxI', 'mH', 'wm', 'ap']
['RV']
['IN', 'ry', 'OkNJ', 'wm', 'ap']
['RV']
None
['Gn', 'tZI']
['Hs', 'RF', 'mD', 'ihLr', 'EEVj', 'EA', 'WN']
['UV']
['Ha']
['UV']
['me', 'RN', 'QZ', 'Ox', 'hVB', 'Sn']
['iy', 'KQ', 'bB']
['AF']
None
['fn', 'jm']
['adO', 'VA']
['bA']
['Gh']
['iu']
['Dg', 'lZ']
['iu']
['Ke', 'Ol']
['pC']
['vl', 'DSy', 'qV']
['CH']
None
None
['lS', 'NB', 'Py', 'uY']
['rR']
['Wa', 'Kj', 'GN', 'NyT', 'Vc']
['kN']
['Utk', 'vY']
['Rfex']
['ly', 'NI', 'Kk']
['lg']
None
['Oo']
['jS', 'wM', 'Zjr', 'ly', 'sd']
None
['hM', 'lb']
None
['WF', 'jK', 'cu', 'wE', 'zJ']
None
None
None
['gH']
['dq']
['eb', 'Ly', 'WFJ', 'OF']
None
None
['ze']
['WP', 'BE'

In [44]:
print(len(scripts))

3076
